In [10]:
import os
os.getcwd()


'C:\\Users\\Sohan\\Downloads\\Movies_dataset_csv'

In [15]:
import pandas as pd

df_movies = pd.read_csv('C:/Users/Sohan/Downloads/Movies_dataset_csv/movies.csv')
df_titles = pd.read_csv('C:/Users/Sohan/Downloads/Movies_dataset_csv/netflix_titles.csv')
df_users = pd.read_csv('C:/Users/Sohan/Downloads/Movies_dataset_csv/users.csv')
df_history = pd.read_csv('C:/Users/Sohan/Downloads/Movies_dataset_csv/watch_history.csv')


In [16]:
# Create 'title_clean' so we can link Movies to Netflix Titles
df_titles['title_clean'] = df_titles['title'].str.lower().str.strip()
df_movies['title_clean'] = df_movies['title'].str.lower().str.strip()

In [17]:
##Clean Duration in netflix_titles (e.g. "90 min" -> 90)
def clean_duration(val):
    if pd.isna(val): return 0
    val = str(val).lower()
    if 'min' in val:
        return int("".join(filter(str.isdigit, val)))
    elif 'season' in val:
        # For simplicity in this project, we can treat seasons differently or just track the number
        return int("".join(filter(str.isdigit, val)))
    return 0

df_titles['duration_clean'] = df_titles['duration'].apply(clean_duration)

In [18]:
# Updated line
df_users = df_users.drop(columns=['email', 'first_name', 'last_name'], errors='ignore')

In [19]:
# 3. CREATE THE MASTER DATAFRAME
print("Merging data...")
# Merge History + Movies
df_master = pd.merge(df_history, df_movies, on='movie_id', how='left', suffixes=('', '_movies'))

# Merge + Users
df_master = pd.merge(df_master, df_users, on='user_id', how='left', suffixes=('', '_users'))

# Merge + Creative (Director/Cast from Netflix Titles)
df_creative = df_titles[['title_clean', 'director', 'cast', 'listed_in', 'duration_clean']]
df_master = pd.merge(df_master, df_creative, on='title_clean', how='left')

# Calculate ROI (Revenue - Budget)
df_master['roi'] = df_master['box_office_revenue'] - df_master['production_budget']
df_master['watch_date'] = pd.to_datetime(df_master['watch_date'])

# 4. DATA HEALTH CHECK & WARNINGS
print("-" * 30)
print(f"Master View Created: {df_master.shape[0]} rows.")
print("-" * 30)

Merging data...
------------------------------
Master View Created: 112546 rows.
------------------------------


In [20]:
# CHECK 1: Financial Data Link
missing_financials = df_master['production_budget'].isnull().sum()
print(f"Rows missing Financial Data: {missing_financials} ({(missing_financials/len(df_master))*100:.1f}%)")

Rows missing Financial Data: 73022 (64.9%)


In [21]:
missing_creative = df_master['director'].isnull().sum()
print(f"Rows missing Director/Cast Data: {missing_creative} ({(missing_creative/len(df_master))*100:.1f}%)")

if (missing_creative/len(df_master)) > 0.8:
    print("\n⚠️ WARNING: High mismatch between 'movies.csv' and 'netflix_titles.csv'.")
    print("Question 9 (Directors) may not have enough data to answer accurately.")
    print("This happens because the movie titles in your financial file don't match the Netflix library file.")

Rows missing Director/Cast Data: 112230 (99.7%)

⚠️ WARNING: High mismatch between 'movies.csv' and 'netflix_titles.csv'.
Question 9 (Directors) may not have enough data to answer accurately.
This happens because the movie titles in your financial file don't match the Netflix library file.


In [22]:
## HANDLING THE FINANCIAL GAP
##creating a specific subset for ROI analysis

df_financials = df_master.dropna(subset=['production_budget', 'box_office_revenue']).copy()
print(f"Financial Analysis Universe: {len(df_financials)} verified records.")

Financial Analysis Universe: 29094 verified records.


In [23]:
# DROPPING THE FAILED CREATIVE LINK
df_master = df_master.drop(columns=['director', 'cast', 'listed_in'], errors='ignore')
print("Dropped incompatible creative columns (Director/Cast).")

Dropped incompatible creative columns (Director/Cast).


In [24]:
df_movies['roi'] = df_movies['box_office_revenue'] - df_movies['production_budget']

In [25]:
df_master['is_high_engagement'] = df_master['progress_percentage'] > 80   ## engagement percentage of user's content choice

In [26]:
print(f"Master Dataframe Shape: {df_master.shape}")  ## printing the final dataset for analysis
df_master.head()

Master Dataframe Shape: (112546, 45)


,session_id,user_id,movie_id,watch_date,device_type,watch_duration_minutes,progress_percentage,action,quality,location_country,...,subscription_plan,subscription_start_date,is_active,monthly_spend,primary_device,household_size,created_at,duration_clean,roi,is_high_engagement
0,session_000001,user_07271,movie_0511,2025-11-13,Tablet,63.9,34.6,completed,HD,USA,...,Basic,18-09-2024,True,25.33,Tablet,3.0,58:10.5,NaN,-778366.0,False
1,session_000002,user_00861,movie_0588,2025-02-26,Laptop,120.1,44.2,started,HD,USA,...,Standard,18-06-2025,True,36.51,Smart TV,NaN,39:11.8,NaN,391287060.0,False
2,session_000003,user_05391,movie_0694,2024-12-15,Desktop,572.1,84.7,started,HD,Canada,...,Basic,16-01-2023,True,17.12,Gaming Console,1.0,06:21.6,NaN,31167586.0,True
3,session_000004,user_05192,movie_0234,2024-09-30,Desktop,395.3,89.9,completed,SD,USA,...,Premium+,27-02-2024,True,37.32,Smart TV,2.0,17:13.7,NaN,NaN,True
4,session_000005,user_05735,movie_0390,2024-08-04,Tablet,14.6,6.2,completed,HD,USA,...,Standard,29-12-2022,True,9.91,Tablet,5.0,32:47.5,NaN,1252192.0,False


In [27]:
# Count null values in one column
df_master["country"].isnull().sum()


0

In [28]:
df_master['country'].value_counts()

country
USA       79039
Canada    33507
Name: count, dtype: int64

In [29]:
# checking  if we have enough data for the next batch
print("-" * 30)
print("Data Readiness for Next Questions:")
print(f"Q4 (Devices): {df_master['device_type'].notna().all()} (100% Coverage)")
print(f"Q6 (Churn): {df_master['is_active'].notna().all()} (100% Coverage)")
print(f"Q8 (Gen Z): {df_master['age'].notna().all()} (100% Coverage)")
print("-" * 30)

------------------------------
Data Readiness for Next Questions:
Q4 (Devices): True (100% Coverage)
Q6 (Churn): True (100% Coverage)
Q8 (Gen Z): False (100% Coverage)
------------------------------


In [30]:
# Q1: The "Vanity Metric" Exposure (High Views, Low Watch Time)
# Uses df_master because we don't need budget data, just views
q1_stats = df_master.groupby('title').agg({
    'session_id': 'count',
    'progress_percentage': 'mean'
}).rename(columns={'session_id': 'view_count'})

# Filter: Top 20% most viewed, but Bottom 20% retention
high_views = q1_stats['view_count'].quantile(0.80)
low_retention = q1_stats['progress_percentage'].quantile(0.20)

vanity_titles = q1_stats[
    (q1_stats['view_count'] > high_views) & 
    (q1_stats['progress_percentage'] < low_retention)
]

print(f"--- Q1: Identified {len(vanity_titles)} 'Clickbait' Titles ---")
print(vanity_titles.sort_values('view_count', ascending=False).head(3))

--- Q1: Identified 17 'Clickbait' Titles ---
              view_count  progress_percentage
title                                        
Last Phoenix         349            47.363578
Ice Secret           346            47.591176
Kingdom War          337            45.911327


In [45]:
# Q2: The "Sunk Cost" Audit (High Budget, Low Completion)
# Uses df_financials (Only rows with verified budget data)
# Filter: Budget > $20M and Completion < 25%
df_financials = df_master.dropna(subset=['production_budget', 'box_office_revenue']).copy()
sunk_cost_failures = df_financials[
    (df_financials['production_budget'] > 20000000) & 
    (df_financials['progress_percentage'] < 25)
].drop_duplicates('title')

print(f"\n--- Q2: Identified {len(sunk_cost_failures)} Budget Flops (>$20M, <25% Completion) ---")
print(sunk_cost_failures[['title', 'production_budget', 'progress_percentage']].head(3))


--- Q2: Identified 36 Budget Flops (>$20M, <25% Completion) ---
              title  production_budget  progress_percentage
196    Love Journey         23460063.0                 18.7
423  Quest Princess         27892159.0                 23.1
527   War Adventure         39546152.0                 23.2


In [32]:
# Q3: Regional ROI Strategy
# Uses df_financials to find which genre makes the most money per country
regional_performance = df_financials.groupby(['location_country', 'genre_primary'])['roi'].median().reset_index()

# Sort to find the top performing genre for each country
top_regional_genres = regional_performance.sort_values(['location_country', 'roi'], ascending=[True, False])
best_per_country = top_regional_genres.groupby('location_country').head(1)

print("\n--- Q3: Best Performing Genre by Country (ROI) ---")
print(best_per_country.head(3))


--- Q3: Best Performing Genre by Country (ROI) ---
   location_country genre_primary         roi
5            Canada         Crime  38477582.0
25              USA         Crime  38477582.0


In [33]:
# Q4: 4K Drop-off on Mobile (Technical Friction)
# We look at 4K content specifically
df_4k = df_master[df_master['quality'] == '4K']
q4_friction = df_4k.groupby('device_type')['progress_percentage'].mean().sort_values()

print("\n--- Q4: 4K Content Completion by Device ---")
print(q4_friction)
if q4_friction.get('Mobile', 100) < q4_friction.get('Desktop', 0) - 10:
    print(">>> INSIGHT: Significant 4K drop-off on Mobile. Recommendation: Disable 4K default on Mobile App.")


--- Q4: 4K Content Completion by Device ---
device_type
Smart TV    49.337909
Desktop     49.740225
Tablet      49.874920
Laptop      49.927459
Mobile      50.310538
Name: progress_percentage, dtype: float64


In [34]:
# Q5: Sleeper Hits (Low Budget, High Rating, High Completion)
# We use df_financials here (only rows with budget data)
# Definition: Budget < $5M, Rating > 4, Completion > 70%
sleeper_hits = df_financials[
    (df_financials['production_budget'] < 5000000) &
    (df_financials['user_rating'] >= 4) &
    (df_financials['progress_percentage'] > 70)
]

print(f"\n--- Q5: Identified {len(sleeper_hits)} 'Sleeper Hits' for Promotion ---")
if not sleeper_hits.empty:
    print(sleeper_hits[['title', 'production_budget', 'user_rating']].head(3))


--- Q5: Identified 473 'Sleeper Hits' for Promotion ---
             title  production_budget  user_rating
135     Last Queen          4303395.0          5.0
471    Dark Legend          1464928.0          4.0
581  First Phoenix           747506.0          5.0


In [35]:
# Q6: Churn Analysis (Last Watched Content)
# Filter for inactive users and find their latest watch session
inactive_users = df_master[df_master['is_active'] == False]
# Sort by date to get the very last interaction
churn_events = inactive_users.sort_values('watch_date', ascending=False).drop_duplicates('user_id')

print("\n--- Q6: Content Linked to Churn (Top Genres before Cancel) ---")
print(churn_events['genre_primary'].value_counts().head(3))


--- Q6: Content Linked to Churn (Top Genres before Cancel) ---
genre_primary
Adventure    103
Animation     95
Sci-Fi        91
Name: count, dtype: int64


In [36]:
# Q7: Ad-Tier Opportunities (Safe Content, Low Revenue)
# Family Friendly (TV-Y, TV-G, G, PG) but Revenue < Budget (ROI < 0)
safe_ratings = ['TV-Y', 'TV-G', 'G', 'PG', 'TV-Y7']
ad_candidates = df_financials[
    (df_financials['rating'].isin(safe_ratings)) &
    (df_financials['content_warning'] == False) &
    (df_financials['roi'] < 0)
]

print(f"\n--- Q7: Found {len(ad_candidates)} Brand-Safe Titles for Ad-Tier Monetization ---")
print(ad_candidates[['title', 'rating', 'roi']].head(3))


--- Q7: Found 4373 Brand-Safe Titles for Ad-Tier Monetization ---
              title rating          roi
0   Dragon Princess      G    -778366.0
42    Fire Princess     PG -191497233.0
46      An Princess   TV-G  -48310603.0


In [37]:
# Q8: Gen-Z Content Strategy (The "False" Fix)
# We filter for users where Age is NOT NaN, then select 18-25
# This handles the "False" coverage issue you saw earlier
df_genz = df_master.dropna(subset=['age'])
df_genz = df_genz[(df_genz['age'] >= 18) & (df_genz['age'] <= 25)]

# Find their favorite genres
genz_favorites = df_genz.groupby('genre_primary')['session_id'].count().sort_values(ascending=False)

print(f"--- Q8: Identified Gen-Z ({len(df_genz)} users) Favorite Genres ---")
print(genz_favorites.head(3))
if not genz_favorites.empty:
    top_genre = genz_favorites.index[0]
    print(f">>> INSIGHT: Gen-Z is heavily indexing on '{top_genre}'. Acquire more of this.")

--- Q8: Identified Gen-Z (14304 users) Favorite Genres ---
genre_primary
Adventure    1028
Animation     905
Sci-Fi        877
Name: session_id, dtype: int64
>>> INSIGHT: Gen-Z is heavily indexing on 'Adventure'. Acquire more of this.


In [38]:
# Q10: TV Series Decay (The "Cancellation Rule")
# We analyze how engagement drops as the 'Number of Seasons' increases
# Logic: Do shows with 6 seasons have lower completion rates than shows with 1 season?
df_series = df_master[df_master['number_of_seasons'].notna()]

season_decay = df_series.groupby('number_of_seasons')['progress_percentage'].mean().sort_values(ascending=False)

print("\n--- Q10: Viewer Retention by Number of Seasons ---")
print(season_decay.head(5))
# Simple Rule-Based Insight
if len(season_decay) > 1:
    drop_off_check = season_decay.iloc[0] - season_decay.iloc[-1]
    if drop_off_check > 15:
        print(f">>> INSIGHT: Long-running series suffer a {drop_off_check:.1f}% drop in retention. Consider capping at 3-4 seasons.")


--- Q10: Viewer Retention by Number of Seasons ---
number_of_seasons
10.0    51.124624
12.0    50.479244
7.0     50.420450
9.0     50.202060
3.0     50.054492
Name: progress_percentage, dtype: float64


In [39]:
# Q11: Market Leadership & Ad Strategy (The "Growth" Question)
# Part A: Which country has the most users? (Using the User Database)
# Note: We use 'country' from the users table, not 'location_country' from history, to get the subscriber base.
top_country = df_users['country'].value_counts().idxmax()
num_users = df_users['country'].value_counts().max()

print(f"\n--- Q11: Market Leader Analysis ({top_country}) ---")
print(f"Top Market: {top_country} with {num_users} subscribers.")

# Part B: What are they watching in that country? (Using Watch History)
country_activity = df_master[df_master['location_country'] == top_country]
top_content_local = country_activity.groupby('title')['watch_duration_minutes'].sum().sort_values(ascending=False)

print(f"Dominant Content in {top_country}:")
print(top_content_local.head(3))

if not top_content_local.empty:
    winner = top_content_local.index[0]
    print(f">>> STRATEGY: Prioritize advertisements for '{winner}' in {top_country}. The audience is already hooked.")


--- Q11: Market Leader Analysis (USA) ---
Top Market: USA with 7204 subscribers.
Dominant Content in USA:
title
A Adventure    29540.5
A Quest        27727.0
Love Battle    24731.3
Name: watch_duration_minutes, dtype: float64
>>> STRATEGY: Prioritize advertisements for 'A Adventure' in USA. The audience is already hooked.


In [40]:
!pip install ydata-profiling --user


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
from ydata_profiling import ProfileReport

In [46]:
profile = ProfileReport(df_financials, title="Content Financial Health Audit", minimal=True)

profile.to_file("financial_audit_report.html")
print("SUCCESS: 'financial_audit_report.html' generated (Word Clouds skipped).")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:01<00:00, 26.91it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

TypeError: asarray() got an unexpected keyword argument 'copy'

In [47]:
# Save the "Brain" to disk so Streamlit can use it
print("Exporting processed data for Streamlit...")

# 1. Save the Master View (For Q1, Q4, Q6, Q8, Q10, Q11)
df_master.to_csv('processed_master.csv', index=False)

# 2. Save the Financial View (For Q2, Q3, Q5, Q7)
df_financials.to_csv('processed_financials.csv', index=False)

print("SUCCESS: 'processed_master.csv' and 'processed_financials.csv' saved.")
print("You are now ready to switch to VS Code.")

Exporting processed data for Streamlit...
SUCCESS: 'processed_master.csv' and 'processed_financials.csv' saved.
You are now ready to switch to VS Code.
